In [86]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from scipy.special import expit
from scipy.optimize import minimize_scalar

import copy

from numpy.linalg import pinv
from sklearn.datasets import load_iris
from scipy.optimize import fmin_bfgs

import multiprocessing as mp

In [87]:
df = pd.read_csv('/Users/jonavatar/Documents/MechineL/processed_data.csv')

In [88]:
df['readmitted'].unique()

array(['>30', 'NO', '<30'], dtype=object)

In [89]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

X = train.drop(['readmitted'], axis=1)
y = train['readmitted']
y.replace({'NO':0,'<30':1,'>30':2}, inplace = True)

X_test = test.drop(['readmitted'], axis=1)
y_test = test['readmitted']
y_test.replace({'NO':0,'<30':1,'>30':2}, inplace = True)

In [90]:
%%time
class BinaryLogisticRegression:
    '''
    method: 'default', 'LineSearchLogisticRegression', 'StochasticLogisticRegression', 
    'MiniBatchStochasticLogisticRegression','HessianBinaryLogisticRegression', 'BFGSBinaryLogisticRegression'
    regularization_metric: 'NA', 'L1', 'L2'
    batch_size: float between 0 to 1, only valid when call 'MiniBatchStochasticLogisticRegression' method
    '''
    def __init__(self, eta, iterations=20, C=0.001, method='default', regularization_metric='NA',batch_size=0.1):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.method = method
        self.regularization_metric = regularization_metric
        self.batch_size = batch_size
        # internally we will store the weights as self.w_ to keep with sklearn conventions
        
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    def objective_function(self,w,X,y,C):
        g = expit(X @ w)
        return -np.sum(np.log(g[y==1]))-np.sum(np.log(1-g[y==0])) + C*sum(w**2) #-np.sum(y*np.log(g)+(1-y)*np.log(1-g))
    def _get_l1_dev(self,val):
        if val > 0:
            return 1
        elif val == 0:
            return 0
        else:
            return -1


    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    def _get_l1_dev(self,x):
        if x>0:
            return 1
        elif x == 0:
            return 0
        else:
            return -1
    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        if self.regularization_metric == 'L2':
            gradient[1:] += -2 * self.w_[1:] * self.C
        elif self.regularization_metric == 'L1':
            tmp = self.w_[1:].copy()
            gradient[1:] += -np.vectorize(self._get_l1_dev)(tmp) * self.C

        return gradient
    
    def _get_gradient_StochasticLogisticRegression(self,X,y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False)# get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        if self.regularization_metric == 'L2':
            gradient[1:] += -2 * self.w_[1:] * self.C
        elif self.regularization_metric == 'L1':
            tmp = self.w_[1:].copy()
            gradient[1:] += -np.vectorize(self._get_l1_dev)(tmp) * self.C

        
        return gradient

    def _get_gradient_MiniBatchStochasticLogisticRegression(self,X,y):
        Idx = np.arange(len(y))
        np.random.shuffle(Idx)
        sample_nbr = int(len(y)*self.batch_size)
        X, y = X[Idx[:sample_nbr]], y[Idx[:sample_nbr]]
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        if self.regularization_metric == 'L2':
            gradient[1:] += -2 * self.w_[1:] * self.C
        elif self.regularization_metric == 'L1':
            tmp = self.w_[1:].copy()
            gradient[1:] += -np.vectorize(self._get_l1_dev)(tmp) * self.C

        return gradient


    def _get_gradient_HessianBinaryLogisticRegression(self,X,y):
        g = self.predict_proba(X,add_bias=False).ravel() # get sigmoid value for all classes
        hessian = X.T @ np.diag(g*(1-g)) @ X - 2 * self.C # calculate the hessian

        ydiff = y-g # get y difference
        gradient = np.sum(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        if self.regularization_metric == 'L2':
            gradient[1:] += -2 * self.w_[1:] * self.C
        elif self.regularization_metric == 'L1':
            tmp = self.w_[1:].copy()
            gradient[1:] += -np.vectorize(self._get_l1_dev)(tmp) * self.C

        
        return pinv(hessian) @ gradient
        
    def line_search_function(self,eta,X,y,w,grad,C):
        wnew = w + grad*eta
        yhat = (1/(1+np.exp(-X @ wnew))) >0.5
        return np.mean((y-yhat)**2)-C*np.mean(wnew**2)
    
    def objective_gradient(self,w,X,y,C):
        g = expit(X @ w)
        ydiff = y-g # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0)
        gradient = gradient.reshape(w.shape)
        if self.regularization_metric == 'L2':
            gradient[1:] += -2 * w[1:] * self.C
        elif self.regularization_metric == 'L1':
            tmp = w[1:].copy()
            gradient[1:] += -np.vectorize(self._get_l1_dev)(tmp) * self.C


        return -gradient

    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    
    def fit(self, X, y):
        if self.method == 'default':
            Xb = self._add_bias(X) # add bias term
            num_samples, num_features = Xb.shape

            self.w_ = np.zeros((num_features,1)) # init weight vector to zeros

            # for as many as the max iterations
            for _ in range(self.iters):
                gradient = self._get_gradient(Xb,y)
                self.w_ += gradient*self.eta # multiply by learning rate 
        elif self.method == 'LineSearchLogisticRegression':
            Xb = self._add_bias(X) # add bias term
            num_samples, num_features = Xb.shape

            self.w_ = np.zeros((num_features,1)) # init weight vector to zeros

            # for as many as the max iterations
            for _ in range(self.iters):
                gradient = self._get_gradient(Xb,y)

                # do line search in gradient direction, using scipy function
                opts = {'maxiter':self.iters/20} # unclear exactly what this should be
                res = minimize_scalar(self.line_search_function, # objective function to optimize
                                      bounds=(self.eta/1000,self.eta*10), #bounds to optimize
                                      args=(Xb,y,self.w_,gradient,self.C), # additional argument for objective function
                                      method='bounded', # bounded optimization for speed
                                      options=opts) # set max iterations

                eta = res.x # get optimal learning rate
                self.w_ += gradient*eta # set new function values
        elif self.method == 'StochasticLogisticRegression':
            Xb = self._add_bias(X) # add bias term
            num_samples, num_features = Xb.shape

            self.w_ = np.zeros((num_features,1)) # init weight vector to zeros

            # for as many as the max iterations
            for _ in range(self.iters):
                gradient = self._get_gradient_StochasticLogisticRegression(Xb,y)
                self.w_ += gradient*self.eta # multiply by learning rate 
        
        elif self.method == 'MiniBatchStochasticLogisticRegression':
            Xb = self._add_bias(X) # add bias term
            num_samples, num_features = Xb.shape
            
            self.w_ = np.zeros((num_features,1)) # init weight vector to zeros

            # for as many as the max iterations
            for _ in range(self.iters):
                gradient = self._get_gradient_MiniBatchStochasticLogisticRegression(Xb,y)
                self.w_ += gradient*self.eta # multiply by learning rate 

        elif self.method == 'HessianBinaryLogisticRegression':
            Xb = self._add_bias(X) # add bias term
            num_samples, num_features = Xb.shape

            self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
            gradient = self._get_gradient_HessianBinaryLogisticRegression(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 
        
        elif self.method == 'BFGSBinaryLogisticRegression':
            Xb = self._add_bias(X) # add bias term
            num_samples, num_features = Xb.shape

            self.w_ = fmin_bfgs(self.objective_function, # what to optimize
                                np.zeros((num_features,1)), # starting point
                                fprime=self.objective_gradient, # gradient function
                                args=(Xb,y,self.C), # extra args for gradient and objective function
                                gtol=1e-03, # stopping criteria for gradient, |v_k|
                                maxiter=self.iters, # stopping criteria iterations
                                disp=False)

            self.w_ = self.w_.reshape((num_features,1))

CPU times: user 37 µs, sys: 8 µs, total: 45 µs
Wall time: 47.9 µs


In [91]:
class MultiClassLogisticRegression:
    '''
    method: 'default', 'LineSearchLogisticRegression', 'StochasticLogisticRegression', 
    'MiniBatchStochasticLogisticRegression','HessianBinaryLogisticRegression', 'BFGSBinaryLogisticRegression'
    regularization_metric: 'NA', 'L1', 'L2'
    batch_size: float between 0 to 1, only valid when call 'MiniBatchStochasticLogisticRegression' method
    processes: number of multiprocesses to use, default value 1
    fit function: instead of fit, use multi_fit, multi_predict_proba, multi_predict
    '''
    def __init__(self, eta, iterations=20, C=0.0001, method='default', 
                 regularization_metric='NA',batch_size=0.1,processes=1):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.classifiers_ = []
        self.method = method
        self.regularization_metric = regularization_metric
        self.batch_size = batch_size
        self.processes = processes


        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
    def bi_fit(self,train_set):
        X,y = train_set[0], train_set[1]
        blr = BinaryLogisticRegression(self.eta,self.iters,self.C,
                                           method=self.method,
                                           regularization_metric=self.regularization_metric,
                                          batch_size=self.batch_size)
        blr.fit(X,y)
        return blr
    def multi_fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y)) # get each unique class value
        num_unique_classes = len(self.unique_)

        train_sets = []
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval).astype(np.int)  # create a binary problem
            train_sets.append([X,y_binary])

        pool = mp.Pool(processes=self.processes)
        self.classifiers_ = pool.map(self.bi_fit, train_sets)


        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def bi_proba(self,predict_set):
        blr, X = predict_set[0], predict_set[1]
        return blr.predict_proba(X).reshape((len(X),1))
    def multi_predict_proba(self,X):
        predict_sets = [[blr,X] for blr in self.classifiers_]
        pool = mp.Pool(processes=self.processes)
        probs = pool.map(self.bi_proba, predict_sets)

        return np.hstack(probs) # make into single matrix
    
    def multi_predict(self,X):
        return np.argmax(self.multi_predict_proba(X),axis=1)

In [92]:

# lr = MultiClassLogisticRegression(eta=0.1,iterations=500,C=0.001,
#                                method='MiniBatchStochasticLogisticRegression',
#                                regularization_metric='L1',
#                               batch_size=0.01,processes=1)
# lr.multi_fit(X,y_test)
# yhat=lr.multi_predict(X_test)
# print('Accuracy of: ',accuracy_score(y_test,yhat))

In [93]:
# from sklearn.linear_model import LogisticRegression
# reg_sklearn = LogisticRegression(multi_class='ovr').fit(X,y)
# y_predict = reg_sklearn.predict(X_test)
# print('Accuracy is',accuracy_score(y_test,y_predict))


In [94]:
methods=['LineSearchLogisticRegression', 'StochasticLogisticRegression', 
    'MiniBatchStochasticLogisticRegression','HessianBinaryLogisticRegression', 'BFGSBinaryLogisticRegression']
metrics=['NA', 'L1', 'L2']
Accuracy_best = 0
performance = np.zeros((len(methods),len(metrics)))
for method_id in range(len(methods)):  
    for metric_id in range(len(metrics)):
        for C_id in range(0,1):
            reg_own = MultiClassLogisticRegression(eta=0.1,iterations=500,C=C_id/1000,
                               method=methods[method_id],
                               regularization_metric=metrics[metric_id],                  
                               batch_size=0.01,processes=1)
            reg_own.multi_fit(X,y)
            Accuracy_own = accuracy_score(y_test,reg_own.predict(X_test))
            performance[method_id][metric_id] = Accuracy_own if performance[method_id][metric_id]<Accuracy_own else performance[method_id][metric_id]
print(performance)



Exception: Data must be 1-dimensional

In [ ]:
import matplot.pyplot as plt
fig, ax = plt.subplots()
im = ax.imshow(performance)

# We want to show all ticks...
ax.set_xticks(np.arange(len(methods)))
ax.set_yticks(np.arange(len(metric)))
# ... and label them with the respective list entries
ax.set_xticklabels(methods)
ax.set_yticklabels(metric)

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=30, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(methods)):
    for j in range(len(matrics)):
        text = ax.text(i, j, performance[i, j],
                       ha="center", va="center", color="w")

fig.tight_layout()
plt.show()